In [ ]:
from __future__ import (absolute_import, division,
                        print_function, unicode_literals)
import numpy as np
import os
import matplotlib.pyplot as plt

try:
    import pickle
except:
    # Python 2.7 ... harumph!
    import cPickle as pickle

from enterprise import constants as const
from enterprise.signals import utils

In [2]:
ephem = 'DE436'
datadir = '/home/pbaker/nanograv/data/'

In [3]:
# read in data pickles
filename = datadir + 'nano11_{}.pkl'.format(ephem)
with open(filename, "rb") as f:
    psrs = pickle.load(f)

In [4]:
p = psrs[18]

tn = p.toas.min()
tx = p.toas.max()
T = tx - tn

t_05 = tn + T*0.05,
t_10 = tn + T*0.10,
t_95 = tx - T*0.05,

i_05 = np.searchsorted(p.toas, t_05)
i_10 = np.searchsorted(p.toas, t_10)
i_95 = np.searchsorted(p.toas, t_95) - len(p.toas)

In [5]:
t_dat = []
for p in psrs:
    toas = p.toas / const.day
    tn = toas.min()
    tx = toas.max()
    T = tx - tn
    t_dat.append({
        'name': p.name,
        'tmin': tn,
        'tmax': tx,
        'Tobs': T,
        't_05': tn + T*0.05,
        't_10': tn + T*0.10,
        't_95': tx - T*0.05,
        'i_05': np.searchsorted(toas, t_05),
        'i_10': np.searchsorted(toas, t_10),
        'i_95': np.searchsorted(toas, t_95) - len(p.toas),
    })
    
    
tmin = np.array([p.toas.min() for p in psrs]) / const.day
tmax = np.array([p.toas.max() for p in psrs]) / const.day
tclip = (tmax - tmin) * 0.05



In [ ]:
for p, t_d in zip(psrs, t_dat):
    # plot TOA resid for this PSR
    toas = p.toas/const.day
    resid = p.residuals / 1.0e-6
    dt = p.toaerrs / 1.0e-6

    plt.figure(figsize=(16,4))
    plt.errorbar(toas, resid, yerr=dt, marker='.', linestyle='none', label=p.name)
    plt.axvline(x=t_d['t_05'], ls=':', color='C1', label='first 5%')
    plt.axvline(x=t_d['t_10'], ls='-.', color='C1', label='first 10%')
    plt.axvline(x=t_d['t_95'], ls='--', color='C1', label='last 5%')
    plt.xlabel('TOA (MJD)')
    plt.ylabel('residual ($u$s)');
    plt.legend();
    plt.show()

In [6]:
p = psrs[3]
U, w = utils.create_quantization_matrix(p.toas, dt=1, nmin=2)

In [15]:
[p.toas[sl]/const.day for sl in utils.quant2ind(U)]

[array([55758.34779284, 55758.34779291, 55758.34779298, 55758.34779305,
        55758.34779312]),
 array([55758.35057527, 55758.35057534, 55758.35057541, 55758.35057548,
        55758.35057556]),
 array([55758.35335683, 55758.3533569 , 55758.35335697, 55758.35335704,
        55758.35335711]),
 array([55758.35613878, 55758.35613885, 55758.35613892, 55758.35613899,
        55758.35613906]),
 array([55758.35892077, 55758.35892084, 55758.35892091, 55758.35892098,
        55758.35892105]),
 array([55758.37558016, 55758.37558016, 55758.37558016]),
 array([55789.32010382, 55789.32010382, 55789.32010382, 55789.32010382,
        55789.32010382, 55789.32010382, 55789.32010382, 55789.32010382,
        55789.32010382, 55789.32010382, 55789.32010382, 55789.32010382,
        55789.32010382, 55789.32010382]),
 array([55789.32289399, 55789.32289399, 55789.32289399, 55789.32289399,
        55789.32289399, 55789.32289399, 55789.32289399, 55789.32289399,
        55789.32289399, 55789.32289399, 55789.3228

In [8]:
epoch = -8
mask = U[:,epoch] > 0
(U[:,epoch] * p.toas/const.day)[mask]

array([57375.93220949, 57375.93220953, 57375.93220963, 57375.93220967])

In [9]:
plt.hist(U[:,4]*p.toas)

(array([8.212e+03, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 5.000e+00]),
 array([0.00000000e+00, 4.81752221e+08, 9.63504442e+08, 1.44525666e+09,
        1.92700888e+09, 2.40876111e+09, 2.89051333e+09, 3.37226555e+09,
        3.85401777e+09, 4.33576999e+09, 4.81752221e+09]),
 <a list of 10 Patch objects>)